In [91]:
# TODO: constraints; cost function? why nega score?; goal is to achieve comparable performance. 

In [1]:
# https://www.kaggle.com/pavelvod/pytorch-starter-solution
!pip install torch

    100% |████████████████████████████████| 71.1MB 657kB/s ta 0:00:011


In [120]:
import utils
import importlib

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [ ]:
# giftCostsNp (5000, 100): [i,j] means the gift cost when i-th family choses j-th day.
row, col = 5000, 100
giftCostsNp = np.zeros((row, col))
familySizesNp = np.zeros((row, 1))

familyDataNp = utils.readDataDf().values

for i in range(row):
    familySizesNp[i] = familyDataNp[i, -1]
    for j in range(col):
        giftCostsNp[i,j] = utils.getCurGift(assignDay=j, 
                                          familyPreferencesList=familyDataNp[i, 1:-1].tolist(), 
                                          familySize=familySizesNp[i])

In [220]:
class Model(nn.Module):
    def __init__(self, giftCostsNp, familySizesNp):
        super().__init__()
        self.giftCostsTc = torch.from_numpy(giftCostsNp).type(torch.float32)
        self.familySizesTc = torch.from_numpy(familySizesNp).type(torch.float32)
        self.assignWeightsTc = torch.nn.Parameter(data=torch.Tensor(5000, 100), requires_grad=True)
        self.assignWeightsTc.data.uniform_(0,5) # weight initialization?
        
    def forward(self):
        assignProbTc = F.softmax(self.assignWeightsTc, dim=1)
#         print(self.giftCostsTc.shape, assignProbTc.shape)
        giftCostTotalTc = (self.giftCostsTc * assignProbTc).sum()
        
        NdTc = (assignProbTc.transpose(0,1) @ self.familySizesTc).sum(axis=1)
#         print(F.softmax(NdTc).max(), F.softmax(NdTc).argmax())
        NdTcCat = torch.zeros(101, dtype=torch.float32)
        NdTcCat[:-1] = NdTc
        NdTcCat[-1] = NdTc[-1]
        
#         print("exponent: ", 0.5+abs(NdTcCat[:-1]-NdTcCat[1:])/50.)
        cleanCostTotalTc = ((NdTc-125.)/400. * NdTc**(0.5+abs(NdTcCat[:-1]-NdTcCat[1:])/50.)).sum()
        
        penaltyCostTotalTc = (torch.relu(125.-NdTc)**2 + torch.relu(NdTc-300.)**2).sum() * 1000000
        return giftCostTotalTc, cleanCostTotalTc, penaltyCostTotalTc, NdTc

In [221]:
# def cost_function():
#     return None
importlib.reload(utils)

<module 'utils' from '/Users/songxu/PycharmProjects/Data_Science/Kaggle/santa-workshop-tour-2019/utils.py'>

In [222]:
model = Model(giftCostsNp, familySizesNp)
best_score = 10e10
best_pos = None

In [ ]:
optimizer = torch.optim.Adam(model.parameters())
for epoch in range(150000):
    optimizer.zero_grad()
    x, y, z, NdTc = model()
#     print(x.shape, y.shape)
    loss = x + y + z
    loss.backward()
    optimizer.step()
    
    if epoch % 1000 == 0:
        # (5000, 100): for each family (keep dim 0), find the most-weighted day
        assignments = model.assignWeightsTc.argmax(dim=1).numpy()+1
#         print(model.assignWeightsTc[0,:])
#         print(model.assignWeightsTc[0,:].max(), model.assignWeightsTc[0,:].argmax())
        
#         print(assignments.max(), assignments.min())


        print("max's:", F.softmax(model.assignWeightsTc, dim=1).max(dim=1).values.data)

        curScore = utils.getTotalScore(assignments, verbose=True)
    
        print("NdTc[:10]:", NdTc[:10].data)
        input("")
    
#         print(x, y, z)
#         print(curScore, z, loss)
#         break
#         pos = model.assignWeightsTc.argmax(1).numpy()
#         a, b, v = cost_function(pos+1)
#         score = a + b
#         if score < best_score:
#             best_score = score
#             best_pos = pos
#         x = np.round(x.item(), 3)
#         y = np.round(y.item(), 3)
#         print(f"{epoch}\t{x}\t{y}\t{score}\t{a}\t{b}\t{v}")

max's: tensor([0.5745, 0.9499, 0.9359,  ..., 0.9948, 0.3971, 0.4943])
NdList[:10] inside getTotalPenalty(): [67, 30, 184, 154, 233, 243, 243, 233, 265, 273]
NdTc[:10]: tensor([125.4360, 135.2435, 187.0518, 178.2332, 218.5107, 246.3865, 234.9274,
        252.8669, 259.5987, 273.9169])

max's: tensor([0.6099, 0.9748, 0.8685,  ..., 0.9974, 0.4094, 0.5429])
NdList[:10] inside getTotalPenalty(): [67, 38, 176, 179, 240, 261, 249, 231, 269, 283]
NdTc[:10]: tensor([125.2969, 157.3845, 189.3737, 197.6497, 230.2791, 251.0608, 232.4649,
        249.0865, 255.8413, 275.3265])

max's: tensor([0.6419, 0.9868, 0.7703,  ..., 0.9986, 0.4250, 0.7851])
NdList[:10] inside getTotalPenalty(): [76, 97, 197, 216, 291, 268, 240, 225, 248, 271]
NdTc[:10]: tensor([125.0205, 217.6847, 205.8865, 235.1218, 260.0786, 256.9468, 233.2404,
        240.7294, 251.0780, 272.1079])

max's: tensor([0.6619, 0.9924, 0.8450,  ..., 0.9991, 0.4365, 0.9069])
NdList[:10] inside getTotalPenalty(): [76, 127, 241, 263, 287, 274, 233,